In [2]:
%matplotlib inline

import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt

In [3]:
linearMesh = uw.mesh.FeMesh_Cartesian('Q2/dPc1', (32,32), (0.,0.), (1.,1.))

In [4]:
constantMesh = linearMesh.subMesh

In [5]:
velocityField = uw.mesh.MeshVariable(linearMesh,2)
velocityField.data[:] = (0.,0.)

In [6]:
pressureField = uw.mesh.MeshVariable(constantMesh,1)
pressureField.data[:] = 0.

In [7]:
# setup freeslip
IWalls = linearMesh.specialSets["MinI_VertexSet"] + linearMesh.specialSets["MaxI_VertexSet"]
JWalls = linearMesh.specialSets["MinJ_VertexSet"] + linearMesh.specialSets["MaxJ_VertexSet"]
freeslip = uw.conditions.DirichletCondition(velocityField, (IWalls, JWalls))

In [8]:
linearMesh.specialSets.keys()

In [9]:
# setup analytic functions
#solA = fn.analytic.SolCx()
# solA = fn.analytic.SolKx()
#solA = fn.analytic.SolKx(1.0, 5.73, 2, 0.7)
solA = fn.analytic.SolM(1.0,1,1,0.6)

In [10]:
# get eta function from solA and take a look
eta = plt.Figure()
eta.Surface(solA.viscosityFn,linearMesh)
eta.show()

In [11]:
# grab the body force from SolA and take a look at vertical component
force = plt.Figure()
force.Surface(solA.bodyForceFn[1], linearMesh)
force.show()

In [12]:
solA.bodyForceFn[1].evaluate((1.0,0.75))


In [13]:
# setup our stokes guy
stokesSystem = uw.systems.Stokes(velocityField,pressureField,solA.viscosityFn,solA.bodyForceFn,conditions=[freeslip,], rtolerance=1.e-5)

In [14]:
# lets solve
stokesSystem.solve()

In [15]:
errMesh=uw.mesh.FeMesh_Cartesian('Q1', (256,256), (0.,0.), (1.,1.))

In [16]:
# lets check FEM solution
velMag = plt.Figure()
velMag.Surface(fn.math.dot(velocityField,velocityField), linearMesh)
velMag.show()

In [17]:
# and analytic
velMagA = plt.Figure()
velMagA.Surface(fn.math.dot(solA.velocityFn,solA.velocityFn), linearMesh)
velMagA.show()

In [18]:
# and difference
velDiff = plt.Figure()
velDiff.Surface(fn.math.dot(solA.velocityFn - velocityField,solA.velocityFn - velocityField), errMesh)
velDiff.show()

In [19]:
# rms
squared = uw.utils.Integral(fn.math.dot(solA.velocityFn-velocityField,solA.velocityFn-velocityField), linearMesh)
area = uw.utils.Integral(1.,linearMesh)
math.sqrt(squared.integrate()[0]/area.integrate()[0])

In [20]:
import numpy as np
N=201
a=np.ndarray(shape=(N,2))
a[:,1]=0.75*np.ones(N)
a[:,0]=np.linspace(0,1,N)
import matplotlib.pyplot as mplt
import matplotlib.pylab as pylab
vy=solA.velocityFn[1].evaluate(a)[:,0]
ax=a[:,0]
vf=velocityField[1].evaluate(a)[:,0]
mplt.plot(ax,vf,label='Numerical')
mplt.plot(ax,vy,label='Exact')
mplt.plot(ax,10000*(vy-vf),label='Error (x 1e+4)')
mplt.legend()
mplt.plot(ax,vf,label='Numerical')
mplt.plot(ax,vy,label='Exact')
mplt.plot(ax,10000*(vy-vf),label='Error (x 1e+4)')
mplt.legend()

In [21]:
vf=velocityField[1].evaluate(a)[:,0]

In [22]:
vf.shape

In [23]:
vy.shape

In [24]:
ax.shape

In [25]:
velx = plt.Figure()
velx.Surface(solA.velocityFn[0], linearMesh)
velx.show()

In [26]:
velz = plt.Figure()
velz.Surface(solA.velocityFn[1], linearMesh)
velz.show()